### Generate knowledge graph and support queries using Neo4j
##### Dependency - install py2neo
##### Prerequisite - Install and have a graph instance running in Neo4j

#### Install Py2neo

In [0]:
! pip install py2neo

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [0]:
from py2neo import Graph
from py2neo import Node, Relationship
import pickle
import pandas as pd

### Load Data

In [0]:
svos = pd.read_csv("svos.csv", names = ['sub', 'rel', 'obj'])
pickle_in = open("entity_dict.pickle","rb")
entities = pickle.load(pickle_in)

### Graph construction

In [0]:
# Connect to Neo4j using corresponding <port:7687> and <password>
graph = Graph("bolt://localhost:7687",password="adbi_capstone")
# graph.delete_all()

# Parse the entities and build the knowledge graph in Neo4j Database
for index, row in svos.iterrows():
    sub, rel, obj = row
    
    sub_node = graph.nodes.match(entities.get(sub, "Object"), name=sub).first()
    obj_node = graph.nodes.match("Object", name=obj).first()
    if not sub_node:
        sub_node = Node(entities.get(sub, "Object"),name = sub)
    if not obj_node:
        obj_node = Node(entities.get(obj, "Object"),name = obj)
    relation = Relationship.type(rel)    
    graph.merge(relation(sub_node, obj_node), entities.get(sub, "Object"), "name" )

## Query

In [0]:
# Cypher queries
def query_actors_names(graph):
    nodes = graph.run("MATCH (:Object {name: 'actor'})-[r]-(m)RETURN r, m")
    names = [ node['m']['name'] for node in nodes]
    print(names)

def query_producers_names(graph):
    nodes = graph.run("MATCH (:Object {name: 'producer'})-[r]-(m)RETURN r, m")
    names = [ node['m']['name'] for node in nodes]
    print(names)


def query_all_nodes(graph):
    nodes = graph.run("MATCH (n)-[r]->(m) RETURN n, r, m")
#     print(nodes.to_table())
    return nodes

def multi_talented(graph):
    nodes = graph.run("MATCH (:Object {name: 'actor'})-[r]-(m)RETURN r, m")
    for node in nodes:
        rels = graph.run("MATCH (:PERSON {name: '%s'})-[r]-(obj)RETURN count(r) as num_rels" % node['m']['name'])
        for rec in rels:
            if rels["num_rels"]>1:
                print(node['m']['name'])
                
def describe(graph, actor):
    nodes = graph.run("MATCH (:PERSON {name: '%s'})-[r]->(m)RETURN r,m" % actor)
    flag=True
    for node in nodes:
        #print(node['m']['name'])
        print(node['r'])
        flag = False
    if flag:
        print("{} not found".format(actor))

query_actors_names(graph)
nodes = query_all_nodes(graph)

for node in nodes:
    print(node['r']) 

['Jon Avery Abrahams', 'Zachary Burr Abel', 'Victor Aaron Ramirez', 'Quinton Aaron', 'Kirk M. Acevedo', 'Murray Abraham October', 'Alexander Bud Abbott', 'Christopher Jacob Abbott', 'Adam Keefe Horovitz', 'Jensen Ross Ackles', 'Yousef AbuTaleb', 'Jay Acovone', 'Walter Abel', 'Jacob Allen Abel', 'Bruce Paul Abbott']
(Rodolfo Perez Acosta)-[:was {}]->(MexicanAmerican character actor)
(Jacob Allen Abel)-[:is {}]->(singer)
(Yousef AbuTaleb)-[:is {}]->(producer)
(Lee William Aaker)-[:is {}]->(child actor)
(Willie Aames)-[:is {}]->(director television producer)
(Willie Aames)-[:is {}]->(screenwriter)
(Jensen Ross Ackles)-[:is {}]->(director)
(which)-[:earned {}]->(Ackles)
(Victor Aaron Ramirez)-[:was {}]->(voice)
(Jon Avery Abrahams)-[:are {}]->(Bobby)
(Cent)-[:is {}]->(television producer entrepreneur)
(Cent)-[:is {}]->(investor)
(Born)-[:began {}]->(selling)
(Costello)-[:were {}]->(comedy duo)
(Abbott)-[:were {}]->(comedy duo)
(Wee Man)-[:is {}]->(one)
(Jason Shannon Acuna)-[:is {}]->(one)

In [0]:
multi_talented(graph)

Jon Avery Abrahams
Victor Aaron Ramirez
Adam Keefe Horovitz
Jensen Ross Ackles
Yousef AbuTaleb
Jacob Allen Abel


In [0]:
query_actors_names(graph)

['Jon Avery Abrahams', 'Zachary Burr Abel', 'Victor Aaron Ramirez', 'Quinton Aaron', 'Kirk M. Acevedo', 'Murray Abraham October', 'Alexander Bud Abbott', 'Christopher Jacob Abbott', 'Adam Keefe Horovitz', 'Jensen Ross Ackles', 'Yousef AbuTaleb', 'Jay Acovone', 'Walter Abel', 'Jacob Allen Abel', 'Bruce Paul Abbott']


### Query about the description of a person

In [0]:
def describe(graph, actor):
    nodes = graph.run("MATCH (:PERSON {name: '%s'})-[r]->(m)RETURN r,m" % actor)
    flag=True
    for node in nodes:
        #print(node['m']['name'])
        print(node['r'])
        flag = False
    if flag:
        print("{} not found".format(actor))

In [0]:
describe(graph, 'Jacob Allen Abel')
describe(graph, 'Adam Keefe Horovitz')

(Jacob Allen Abel)-[:is {}]->(singer)
(Jacob Allen Abel)-[:is {}]->(actor)
(Adam Keefe Horovitz)-[:is {}]->(actor)
(Adam Keefe Horovitz)-[:is {}]->(rapper guitarist)


(Adam Keefe Horovitz)-[:is {}]->(actor)
(Adam Keefe Horovitz)-[:is {}]->(rapper guitarist)


In [0]:
query_producers_names(graph)

['Yousef AbuTaleb']
